# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model = model_LDA(lattice, atoms, positions, symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-14);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.235094935346                   -0.50    7.0
  2   -7.250253088284       -1.82       -1.40    1.0
  3   -7.251166752325       -3.04       -2.01    2.0
  4   -7.251072913205   +   -4.03       -1.95    2.0
  5   -7.251329107967       -3.59       -2.65    1.0
  6   -7.251338238358       -5.04       -3.19    2.0
  7   -7.251338738656       -6.30       -3.70    1.0
  8   -7.251338789031       -7.30       -4.11    1.0
  9   -7.251338797904       -8.05       -4.74    3.0
 10   -7.251338798651       -9.13       -5.22    2.0
 11   -7.251338798698      -10.33       -5.69    2.0
 12   -7.251338798704      -11.23       -6.32    2.0
 13   -7.251338798704      -12.46       -6.74    2.0
 14   -7.251338798705      -13.37       -7.07    3.0
 15   -7.251338798705      -14.27       -7.66    1.0


Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.0913775982178309
[ Info: Arnoldi iteration step 2: normres = 0.44974471143114614
[ Info: Arnoldi iteration step 3: normres = 0.6507817516839228
[ Info: Arnoldi iteration step 4: normres = 0.23745905963664166
[ Info: Arnoldi iteration step 5: normres = 0.5410324602624126
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (1.02e-02, 8.16e-02, 4.85e-01, 2.25e-01, 1.48e-02)
[ Info: Arnoldi iteration step 6: normres = 0.29521755746441974
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (2.42e-03, 1.13e-01, 2.25e-01, 1.22e-01, 8.31e-02)
[ Info: Arnoldi iteration step 7: normres = 0.07219116006779588
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (8.45e-05, 8.99e-03, 1.27e-02, 3.67e-02, 5.03e-02)
[ Info: Arnoldi iteration step 8: normres = 0.10817706835432732
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (3.92e